In [1]:
import requests
import bs4
import time
import json
import os.path

In [82]:
if not os.path.exists('repos.json'):
    with open('repos.json', 'w') as fp:
        pass

with open('repos.json') as fp:
    vans = [json.loads(line) for line in fp]
crawled_vans = set([van['url'] for van in vans])

In [3]:
roots = ['http://www.repo.com/browse/domestic_cargo_vans_cube_vans/?sort=date-',
         'http://www.repo.com/browse/import_cargo_vans_cube_vans/?sort=date-']

In [4]:
responses = [requests.get(root) for root in roots]


In [6]:
soups = [bs4.BeautifulSoup(response.text) for response in responses]

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [98]:
def startsWithListing(x): 
    return x and x.startswith('/listing')
postings = [tag['href'] for soup in soups for tag in soup.find_all('a', href = startsWithListing)]
urls = ['http://www.repo.com' + posting for posting in postings]
urls = set([url for url in urls if url not in crawled_vans])

In [99]:
vans = [requests.get(url) for url in urls]

In [100]:
vans[0].text

'<!DOCTYPE html>\n<html>\n<!--\n\n\n  This website is Copyright (c) 2016 Repo.com.  All Rights Reserved.\n\n  Except as required for client functionality, no part of this site may be\n  downloaded, reproduced, modified, used to create derivative works, or\n  transmitted in any form or by any means, except as permitted under Section\n  107 of the 1976 U.S. Copyright Act. Visitors will not attempt to reverse\n  engineer, disassemble, decompose, amend or modify or similarly manipulate\n  all or any portion of Repo.com.\n\n\n-->\n<head>\n<title>Repo.com</title>\n\n\n\n<meta name="Copyright" content="Copyright (c) 2016 Repo.com. All Rights Reserved." />\n<meta name="viewport" content="width=device-width, initial-scale=1" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" /><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}r

In [145]:
def startsWithAsset(x): 
    return x and x.startswith('/assets')

def extract_posting(van_soup):
    column = van_soup.find('div', {'class':'col-md-6'})
    table = van_soup.find('table', {'class': 'details table table-striped'})
    posting = {}
    try:
        posting = {key.text[:-1]:' '.join(value.text.split()) for key, value in zip (table.find_all('th'), table.find_all('td'))}
        posting['images'] = ['http://www.repo.com%s' % tag['src'] for tag in van_soup.find_all('img', src=startsWithAsset)]
        posting['images'] = posting['images'][1:]
        posting['title'] = column.find('h3').text
        posting['price'] = column.find('h4').text.split('or')[0].strip()[1:].replace(',', '').replace('.00', '')
        posting['odometer'] = posting['Mileage'].split(' km')[0]
        posting['body'] = column.find('p', {'class':'summary'}).text.strip()
        posting['city'] = (van_soup
                       .find_all('div', {'class':'panel-body'})[2]
                       .find_all('div', {'class':'col-md-6'})[2]
                       .find('p')
                       .text
                       .split('\n')[1].split()[-1])
    except Exception as e:
        print ("Error occured:", e)
    
    return posting
    
def crawl_vans(vans, urls):
    for van, url in zip(vans, urls):
        print (url)
        soup = bs4.BeautifulSoup(van.text)
        posting = extract_posting(soup)
        posting['url'] = url
        posting['date'] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        posting['deadlink'] = False
        posting['crawler'] = 'Repos'
        with open('repos.json', 'a') as fp:
            fp.write(json.dumps(posting) + '\n')

In [146]:
crawl_vans(vans, urls)

http://www.repo.com/listing/5346
http://www.repo.com/listing/5471
http://www.repo.com/listing/5077
http://www.repo.com/listing/4745
http://www.repo.com/listing/5474


/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


http://www.repo.com/listing/5167
http://www.repo.com/listing/5469
http://www.repo.com/listing/5476
http://www.repo.com/listing/5260
http://www.repo.com/listing/5396
Error occured: 'NoneType' object has no attribute 'find_all'
http://www.repo.com/listing/5429
http://www.repo.com/listing/5304
http://www.repo.com/listing/5010
Error occured: 'NoneType' object has no attribute 'find_all'
http://www.repo.com/listing/4982
http://www.repo.com/listing/5464
http://www.repo.com/listing/5241
http://www.repo.com/listing/5372
http://www.repo.com/listing/5460
http://www.repo.com/listing/5458
Error occured: 'NoneType' object has no attribute 'find_all'
http://www.repo.com/listing/5225


In [103]:
len(vans)

20